# 1. Импорт библиотек и загрузка данных

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score,
    confusion_matrix,
    classification_report
)
from sklearn.feature_selection import SelectKBest, f_classif

# Загружаем данные (предполагаем, что train.csv и test.csv уже загружены)
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Объединяем для EDA (если test_data без целевой переменной)
data = train_data.copy()

EmptyDataError: No columns to parse from file

# 2. Разведочный анализ данных (EDA)

### 2.1. Общая информация

In [5]:
print("Размер датасета:", data.shape)
print("\nПервые 5 строк:")
display(data.head())

print("\nИнформация о данных:")
data.info()

print("\nОписательная статистика:")
display(data.describe().T)

Размер датасета: (400000, 2)

Первые 5 строк:


,ID,class
0,600000,0
1,600001,0
2,600002,0
3,600003,0
4,600004,0



Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   ID      400000 non-null  int64
 1   class   400000 non-null  int64
dtypes: int64(2)
memory usage: 6.1 MB

Описательная статистика:


,count,mean,std,min,25%,50%,75%,max
ID,400000.0,799999.5,115470.198175,600000.0,699999.75,799999.5,899999.25,999999.0
class,400000.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0


### 2.2. Проверка на пропуски

In [6]:
print("Пропущенные значения:")
print(data.isnull().sum())

Пропущенные значения:
ID       0
class    0
dtype: int64


### 2.3. Распределение целевой переменной

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(x='target', data=data)
plt.title("Распределение классов (0 = здоров, 1 = ССЗ)")
plt.show()

print("\nБаланс классов:")
print(data['target'].value_counts(normalize=True))

### 2.4. Визуализация числовых признаков

In [ ]:
numerical_features = ['Age', 'Resting_blood_pressure', 'Serum_cholestoral', 'Maximum_heart_rate_achieved', 'Oldpeak']
data[numerical_features].hist(bins=20, figsize=(12, 8))
plt.tight_layout()
plt.show()

# 3. Корреляционный анализ

In [ ]:
plt.figure(figsize=(12, 8))
corr_matrix = data.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Тепловая карта корреляции")
plt.show()

# Вывод самых значимых корреляций с целевой переменной
print("\nКорреляция с целевой переменной:")
print(corr_matrix['target'].sort_values(ascending=False))

# 4. Подготовка данных

### 4.1. Разделение на признаки и целевую переменную

In [ ]:
X = data.drop(['id', 'target'], axis=1)
y = data['target']

# Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### 4.2. Масштабирование данных

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. Обучение моделей

### 5.1. Логистическая регрессия (Logistic Regression)

In [ ]:
lr = LogisticRegression(max_iter=1000, class_weight='balanced')
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

print("\nLogistic Regression:")
print(classification_report(y_test, y_pred_lr))
print("ROC-AUC:", roc_auc_score(y_test, lr.predict_proba(X_test_scaled)[:, 1]))

### 5.2. Случайный лес (Random Forest)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

print("\nRandom Forest:")
print(classification_report(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test_scaled)[:, 1]))

### 5.3. Нейронная сеть (Neural Network)

In [ ]:
nn = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
nn.fit(X_train_scaled, y_train)
y_pred_nn = nn.predict(X_test_scaled)

print("\nNeural Network:")
print(classification_report(y_test, y_pred_nn))
print("ROC-AUC:", roc_auc_score(y_test, nn.predict_proba(X_test_scaled)[:, 1]))

# 6. Сравнение моделей

In [ ]:
models = {
    'Logistic Regression': y_pred_lr,
    'Random Forest': y_pred_rf,
    'Neural Network': y_pred_nn
}

results = []
for name, preds in models.items():
    results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, preds),
        'Precision': precision_score(y_test, preds),
        'Recall': recall_score(y_test, preds),
        'F1-Score': f1_score(y_test, preds),
        'ROC-AUC': roc_auc_score(y_test, preds)
    })

results_df = pd.DataFrame(results)
display(results_df.sort_values(by='F1-Score', ascending=False))